## Importação das Bibiliotecas

In [14]:
from pyspark.sql import SparkSession
import pandas as pd
from sqlalchemy import create_engine
import pyodbc
import pyspark.sql.functions as F


## Criando uma SparkSession

In [15]:
spark = SparkSession.builder.appName("LerDadosEnem").getOrCreate()


## Processo de Leitura do CSV de Participantes


In [16]:
#Path dos arquivos

caminho_arquivo_2022 = r'E:\Estudos\SQL\Datasets\ENEM\2022\microdados_enem_2022\DADOS\MICRODADOS_ENEM_2022.csv'
caminho_arquivo_2021 = r'E:\Estudos\SQL\Datasets\ENEM\2021\microdados_enem_2021\DADOS\MICRODADOS_ENEM_2021.csv'
caminho_arquivo_2020 = r'E:\Estudos\SQL\Datasets\ENEM\2020\microdados_enem_2020\DADOS\MICRODADOS_ENEM_2020.csv'
caminho_arquivo_2019 = r'E:\Estudos\SQL\Datasets\ENEM\2019\microdados_enem_2019\DADOS\MICRODADOS_ENEM_2019.csv'

In [17]:
#Função Paa ler os arquivos CSV
def lerCSV(spark, caminho_arquivo):
    df = spark.read.csv(caminho_arquivo, sep=';', header=True, inferSchema=True, encoding='ISO-8859-1')
    colunas_selecionadas = [F.col(df.columns[0])] + [F.col(df.columns[i]) for i in range(23, 44)]
    return df.select(colunas_selecionadas)


In [18]:
##Chama função quer ler o arquivo CSV e Transforma para DF.SPARK

dfDadosProvas22 = lerCSV(spark, caminho_arquivo_2022)
dfDadosProvas21 = lerCSV(spark, caminho_arquivo_2021)
dfDadosProvas20 = lerCSV(spark, caminho_arquivo_2020)
dfDadosProvas19 = lerCSV(spark, caminho_arquivo_2019)



In [19]:
#Unir os DF em um só 

dfDadosProvas = dfDadosProvas22.union(dfDadosProvas21).union(dfDadosProvas20).union(dfDadosProvas19)


In [20]:
dfDadosProvas.show()

+------------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|NU_INSCRICAO|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|CO_PROVA_CN|CO_PROVA_CH|CO_PROVA_LC|CO_PROVA_MT|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|     TX_RESPOSTAS_CN|     TX_RESPOSTAS_CH|     TX_RESPOSTAS_LC|     TX_RESPOSTAS_MT|TP_LINGUA|      TX_GABARITO_CN|      TX_GABARITO_CH|      TX_GABARITO_LC|      TX_GABARITO_MT|
+------------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------

In [21]:
dfPreencher = dfDadosProvas.fillna(-2)  # Preenche com -2(sem ref), por exemplo


In [22]:
dfPreencher.show()

+------------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|NU_INSCRICAO|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|CO_PROVA_CN|CO_PROVA_CH|CO_PROVA_LC|CO_PROVA_MT|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|     TX_RESPOSTAS_CN|     TX_RESPOSTAS_CH|     TX_RESPOSTAS_LC|     TX_RESPOSTAS_MT|TP_LINGUA|      TX_GABARITO_CN|      TX_GABARITO_CH|      TX_GABARITO_LC|      TX_GABARITO_MT|
+------------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------

# Salvar em SQL 
Salvar os dados em uma tabela SQL


## Parametrizando a conexão do Banco SQL SERVER Usando JDBC

In [23]:
server_name = "jdbc:sqlserver://NOTEBOOK-LEO:1433"
database_name = "Enem"
table_name = "ProvasObjetivas"
username = "bi_leonardobo"
password = "0000"
url = server_name + ";" + "databaseName=" + database_name + ";"
print(url)


jdbc:sqlserver://NOTEBOOK-LEO:1433;databaseName=Enem;


In [24]:
# Nome da nova tabela que será criada
nomeTabela = table_name

In [28]:
# Configurações do banco de dados em ODBC para dropar
server_name = "NOTEBOOK-LEO\\SQLEXPRESS"
 
# Conecte-se ao banco de dados usando pyodbc
conn = pyodbc.connect(f"Driver={{SQL Server}};Server={server_name};Database={database_name};UID={username};PWD={password}")

# Crie um cursor
cursor = conn.cursor()

# Execute a operação DROP TABLE na tabela 'ProvasObjetivas'
truncate_query = f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name}"
cursor.execute(truncate_query)

# Commit as alterações no banco de dados
conn.commit()

In [29]:
# Escreva os novos dados em 'ProvasObjetivas' usando a conexão JDBC
dfPreencher.write \
        .format("jdbc") \
        .option("url", url) \
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")\
        .option("ssl", "true")\
        .option("trustServerCertificate", "true")\
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password)\
        .option("mode", "overwrite").save()
  


Py4JJavaError: An error occurred while calling o356.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 50 in stage 22.0 failed 1 times, most recent failure: Lost task 50.0 in stage 22.0 (TID 281) (Notebook-Leo executor driver): com.microsoft.sqlserver.jdbc.SQLServerException: Não foi possível alocar espaço para o objeto 'dbo.ProvasObjetivas' do banco de dados 'Enem' porque o grupo de arquivos 'PRIMARY' está cheio. Crie espaço em disco excluindo arquivos desnecessários, removendo objetos do grupo de arquivos, adicionando arquivos ao grupo ou definido o aumento automático para arquivos existentes do grupo.
	at com.microsoft.sqlserver.jdbc.SQLServerException.makeFromDatabaseError(SQLServerException.java:255)
	at com.microsoft.sqlserver.jdbc.SQLServerStatement.getNextResult(SQLServerStatement.java:1695)
	at com.microsoft.sqlserver.jdbc.SQLServerPreparedStatement.doExecutePreparedStatementBatch(SQLServerPreparedStatement.java:2978)
	at com.microsoft.sqlserve

In [ ]:
print(f'Tabela {nomeTabela} criada com sucesso no banco de dados.')


Tabela ProvasObjetivas criada com sucesso no banco de dados.


In [ ]:
spark.stop()